<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/M11_Lab2_Multi_Agent_Investment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💹 <span style="color:#2c3e50;">Advanced CrewAI:</span> <span style="color:#16a085;">Multi-Agent Investment Analysis</span> with Delegation & RAG

## 📘 <span style="color:#34495e;">Lab Overview</span>

You already know the basics of CrewAI — now let’s explore its **advanced features** that unlock real-world power.  
In this lab, you'll build a **sophisticated investment analysis system** demonstrating:

### 🎯 <span style="color:#2980b9;">Advanced Features You'll Master</span>
- 🧠 <strong>Agent Delegation</strong> – Let agents automatically assign work to specialists  
- 📄 <strong>RAG Integration</strong> – Analyze uploaded financial documents with AI  
- 📈 <strong>Real-time Data</strong> – Combine live market data with AI analysis  
- 📝 <strong>Professional Output</strong> – Transform messy AI responses into clean reports

### 💼 <span style="color:#8e44ad;">What You're Building</span>
A **4-agent investment team** that works like a real Wall Street firm:  
- The **Portfolio Manager** delegates to specialists  
- The **Research Analyst** reads your uploaded documents  
- The team produces **investment recommendations** using live data

### 🔥 <span style="color:#c0392b;">Why This Matters</span>
These patterns — delegation, RAG, real-time integration — are **essential for building production-grade AI systems** that can handle complex, multi-step workflows across any domain.


In [ ]:
# +++++ 📦 Package Installation
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Install required packages for advanced CrewAI features and financial data integration

!pip install -q "crewai<=1.4.0" "crewai-tools<=1.4.0" langchain-openai yfinance plotly "qdrant-client<=1.15.0"
print("✅ All packages installed successfully!")

# 📌 Package Explanations:
# - crewai: Core library to define and manage multi-agent AI workflows.
# - crewai-tools: Adds tools and enhancements to improve agent capabilities in CrewAI.
# - langchain-openai: Enables integration of OpenAI LLMs with LangChain for natural language processing.
# - yfinance: Used to fetch real-time and historical financial data from Yahoo Finance.
# - plotly: Enables creation of interactive and visually appealing charts for data analysis.

# Start time tracking (put this at the beginning of your lab)
import time
from datetime import datetime
start_time = time.time()

In [ ]:
# +++++ 🎨 Pretty Print Utility
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Create styled message boxes for better visual output in Colab

from IPython.display import display, HTML

def pretty_print(text, title="ℹ️ Info", theme="blue"):
    """Displays a styled message box with optional color themes: blue, red, or yellow."""

    themes = {
        "blue": {"color": "#1e4b8f", "background": "#f0f6ff"},
        "red": {"color": "#c62828", "background": "#ffebee"},
        "yellow": {"color": "#b26a00", "background": "#fff8e1"}
    }

    style = themes.get(theme.lower(), themes["blue"])
    formatted_text = text.replace('\n', '<br>')

    display(HTML(f"""
    <div style="border-left: 5px solid {style['color']}; padding: 12px 16px; background-color: {style['background']};
                border-radius: 6px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; margin: 10px 0;">
        <strong style="color: {style['color']}; font-size: 16px;">{title}</strong><br>
        <span style="font-size: 14px; color: #333;">{formatted_text}</span>
    </div>
    """))

print("🎨 Pretty print utility ready!")

In [ ]:
# +++++ 🔑 API Setup & Authentication
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Secure API key setup from Google Colab secrets

try:
    from google.colab import userdata
    import os
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    pretty_print("🔐 OpenAI API key successfully loaded. You're authenticated and ready to go!", "✅ API Key Setup", "blue")
except:
    pretty_print("⚠️ OpenAI API key not found. Please set it in Colab ➤ More ➤ Secrets before running the lab.", "❌ Missing API Key", "red")


## 🛠️ Advanced Tools & Agent Setup

Now we'll set up the advanced CrewAI components that make this system powerful:

**🔧 Specialized Tools:**
- **Financial Web Scrapers** - Extract live data from Yahoo Finance and MarketWatch
- **RAG File Reader** - Analyze your uploaded financial documents (PDFs, reports)
- **Web Search** - General purpose research capabilities

**👥 The 4-Agent Investment Team:**
1. **📊 Portfolio Manager** - Has delegation powers, coordinates the entire analysis
2. **📰 Market Analyst** - Scrapes financial websites for current news and sentiment
3. **📚 Research Analyst** - Uses RAG to read and analyze your uploaded documents
4. **💹 Trading Strategist** - Synthesizes everything into actionable recommendations

**🔥 Key Advanced Features:**
- **Delegation**: Portfolio Manager can automatically assign tasks to specialists
- **RAG**: Research Analyst reads YOUR uploaded files (earnings reports, SEC filings)
- **Specialization**: Each agent has specific tools and expertise areas

In [ ]:
# +++++ 🛠️ Import Libraries & Initialize Tools
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Import CrewAI framework and set up specialized tools for financial analysis

from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
# Built-in tools for web search, web scraping, and file reading
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool, FileReadTool

# FINANCIAL DATA SCRAPING TOOLS:
yahoo_finance_scraper = ScrapeWebsiteTool(website_url='https://finance.yahoo.com')    # Live stock prices
marketwatch_scraper = ScrapeWebsiteTool(website_url='https://www.marketwatch.com')    # Market news
web_search = WebsiteSearchTool()                                                       # General web search

# RAG (Retrieval-Augmented Generation) TOOL:
file_reader = FileReadTool()  # Reads PDFs, text files, and documents you upload

# AI MODEL CONFIGURATION:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)  # Lower temp = more consistent output

pretty_print("Investment tools locked and loaded!", "🔧 Tools Ready", "blue")


In [ ]:
# +++++ 👥 Create Specialized AI Agents with Delegation
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Create 4 specialized AI agents that work together like a real investment team

# AGENT 1: PORTFOLIO MANAGER (THE BOSS)
portfolio_manager = Agent(
    role="Senior Portfolio Manager",
    goal="Make strategic investment decisions by coordinating research team",
    backstory="You're a seasoned Wall Street portfolio manager with 15+ years experience. You delegate research tasks to specialists and make final investment calls based on comprehensive analysis.",
    llm=llm,
    tools=[web_search],
    allow_delegation=True,  # 🔥 DELEGATION POWER! This agent can delegate tasks to others
    max_delegation=3,       # Can delegate up to 3 tasks at once
    verbose=False
)
# 📌 This agent acts as the team leader, assigning tasks and integrating all outputs to make final investment decisions.
# Think of it as the executive strategist who oversees the whole operation and ensures every agent’s work aligns with the firm’s goals.

# AGENT 2: MARKET NEWS ANALYST
market_analyst = Agent(
    role="Market News Analyst",
    goal="Track real-time market news and sentiment for specific stocks",
    backstory="You're a former financial journalist who now specializes in analyzing market news, earnings reports, and sentiment. You have your finger on the pulse of Wall Street.",
    llm=llm,
    tools=[yahoo_finance_scraper, marketwatch_scraper, web_search],  # Has access to financial websites
    verbose=False
)
# 📰 This agent monitors live financial news and trends from trusted sources to detect any signals or events that may impact stock prices.
# It plays a critical role in sentiment analysis and contextual understanding of market movement.

# AGENT 3: RESEARCH DOCUMENT ANALYST (RAG SPECIALIST)
research_analyst = Agent(
    role="Research Document Analyst",
    goal="Analyze financial documents, earnings reports, and research files",
    backstory="You're a CFA charterholder who excels at digging through financial documents, SEC filings, and research reports to find hidden insights and key metrics.",
    llm=llm,
    tools=[file_reader, web_search],  # Can read your uploaded files + web search
    verbose=False
)
# 📄 This agent is the RAG powerhouse — it reads user-uploaded documents and extracts valuable insights.
# It’s ideal for deep analysis of PDFs, reports, and any offline data that supports investment decisions.

# AGENT 4: TRADING STRATEGIST
trading_strategist = Agent(
    role="Trading Strategist",
    goal="Synthesize all research into actionable BUY/SELL/HOLD recommendations",
    backstory="You're a quantitative analyst who combines technical analysis, fundamental analysis, and market sentiment to create clear, actionable trading strategies with specific price targets.",
    llm=llm,
    tools=[web_search],
    verbose=False
)
# 💹 This is the final decision-maker who converts all research into real trading signals.
# It crafts buy/sell/hold strategies with target prices by balancing risk, trend, and fundamental indicators.

pretty_print("💼 Investment dream team assembled!\n📊 Portfolio Manager (Boss)\n📰 Market Analyst\n📚 Research Analyst\n💹 Trading Strategist", "👥 Team Ready", "blue")


## 📋 Task Definition & Workflow Design

Here's where the advanced CrewAI features really shine. We'll create tasks that demonstrate:

**🎯 Delegation in Action:**
The Portfolio Manager doesn't do the work directly - instead, it **delegates** specific tasks to the right specialists and then **coordinates** their findings into a final recommendation.

**📚 RAG Implementation:**
The Research Analyst can read and analyze any financial documents you upload (earnings reports, SEC filings, research papers) and extract key insights that wouldn't be available through web search alone.

**🔄 Workflow Process:**
1. Portfolio Manager **delegates** news analysis to Market Analyst
2. Portfolio Manager **delegates** document analysis to Research Analyst  
3. Portfolio Manager **delegates** strategy creation to Trading Strategist
4. Portfolio Manager **synthesizes** all findings into executive summary

**💡 Why This Architecture Works:**
Just like a real investment firm, specialization + coordination produces better results than any single agent trying to do everything.

In [ ]:
# +++++ 📋 Define Agent Tasks with Specific Formats
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Create specific tasks for each agent with exact output formats to avoid messy results

def create_investment_tasks(stock_symbol):
    """
    Creates 4 specialized tasks that demonstrate delegation and RAG capabilities.
    Each task has specific output format to prevent messy results.
    """

    # TASK 1: PORTFOLIO MANAGER COORDINATION TASK (DELEGATION)
    coordination_task = Task(
        description=f"""As Portfolio Manager, coordinate investment analysis for {stock_symbol}.

        DELEGATE to your team and create EXECUTIVE SUMMARY in this EXACT format:

        🏢 {stock_symbol} INVESTMENT ANALYSIS
        💰 RECOMMENDATION: [BUY/SELL/HOLD]
        🎯 Target Price: $[X.XX]
        📉 Stop Loss: $[X.XX]
        ⏰ Timeframe: [Short/Medium/Long term]

        KEY INSIGHTS (3 bullet points max):
        • [Key point 1]
        • [Key point 2]
        • [Key point 3]

        ⚠️ RISKS: [Main risk in 1 sentence]

        Keep it SHORT and ACTIONABLE. No paragraphs!""",
        agent=portfolio_manager,
        expected_output=f"Clean executive summary for {stock_symbol} with exact format"
    )

    # TASK 2: MARKET NEWS ANALYSIS TASK (WEB SCRAPING)
    market_task = Task(
        description=f"""Find latest news for {stock_symbol} and summarize in EXACTLY this format:

        📰 MARKET NEWS ({stock_symbol})
        • [Latest news headline 1]
        • [Latest news headline 2]
        • [Latest news headline 3]

        📊 SENTIMENT: [Positive/Negative/Neutral] - [Why in 1 sentence]

        Keep it SHORT! Max 4 lines total.""",
        agent=market_analyst,
        expected_output=f"Short news summary for {stock_symbol} in exact format"
    )

    # TASK 3: DOCUMENT ANALYSIS TASK (RAG)
    research_task = Task(
        description=f"""Analyze {stock_symbol} financials and provide EXACTLY this format:

        📊 FINANCIAL HEALTH ({stock_symbol})
        • Revenue: [Growing/Declining/Stable]
        • Profit: [Strong/Weak/Average]
        • Debt: [Low/Medium/High]

        💡 KEY METRIC: [Most important number]

        Keep it SHORT! Max 4 lines total.""",
        agent=research_analyst,
        expected_output=f"Short financial summary for {stock_symbol} in exact format"
    )

    # TASK 4: TRADING STRATEGY TASK (SYNTHESIS)
    strategy_task = Task(
        description=f"""Create trading strategy for {stock_symbol} in EXACTLY this format:

        💹 TRADING STRATEGY ({stock_symbol})
        >>  Today's Price ($[X.XX])
        🎯 Action: [BUY/SELL/HOLD]
        💰 Entry Price: $[X.XX]
        🚀 Target: $[X.XX]
        🛑 Stop Loss: $[X.XX]

        📈 WHY: [Reason in 1 sentence]

        Keep it SHORT! Max 5 lines total.""",
        agent=trading_strategist,
        expected_output=f"Short trading strategy for {stock_symbol} in exact format"
    )

    return coordination_task, market_task, research_task, strategy_task

print("📋 Task templates created - ready for delegation!")

## 🚀 Crew Assembly & Advanced Output Processing

The final step brings everything together with two key advanced features:

**🎯 Multi-Agent Orchestration:**
The `analyze_stock()` function creates a crew where agents can delegate tasks to each other, work in parallel when possible, and coordinate their findings automatically.

**📊 Real-Time Data Integration:**
The `clean_investment_output()` function demonstrates how to enhance AI analysis with live data - it fetches current stock prices, recent trading history, and key metrics from financial APIs, then formats everything into a professional investment report.

**💡 Why This Approach Works:**
- **Delegation** ensures the right specialist handles each task
- **RAG** incorporates your private documents into the analysis  
- **Real-time data** keeps recommendations current and actionable
- **Clean formatting** transforms messy AI output into professional reports

This pattern can be adapted for any domain where you need specialized AI agents working with both private documents and live data sources.

In [ ]:
# +++++ 🚀 Crew Assembly & Execution Function
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Create main function that assembles all agents into a working crew

def analyze_stock(stock_symbol):
    """
    Main function that runs the complete investment analysis workflow.
    Demonstrates multi-agent collaboration with delegation and real-time data integration.
    """

    # Create tasks for all 4 agents
    coord_task, market_task, research_task, strategy_task = create_investment_tasks(stock_symbol)

    # Assemble agents into crew - enables delegation between agents
    investment_crew = Crew(
        agents=[portfolio_manager, market_analyst, research_analyst, trading_strategist],
        tasks=[coord_task, market_task, research_task, strategy_task],
        verbose=False,          # Keeps output clean
        max_iter=5,            # Allows for delegation loops
        output_log_file=False  # Prevents messy log files
    )

    pretty_print(f"Analyzing {stock_symbol}...", "💹 Processing", "yellow")

    # Execute crew - this is where delegation happens automatically
    result = investment_crew.kickoff()

    return result

pretty_print("✅ Main analysis logic is ready to run!", title="🟢 CrewAI Initialized")

In [ ]:
# +++++ 🎯 Execute Advanced CrewAI Analysis
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Execute the complete analysis for Apple stock
print("Running advanced CrewAI analysis with delegation and real-time data...")
result = analyze_stock('AAPL')
print(result)

pretty_print("✅ Analysis complete! Your AI investment team used delegation to coordinate specialists, integrated real-time market data, and produced a professional investment report.", "🎊 Success", "blue")

In [ ]:
# +++++ 📊 HTML Investment Report (Dropbox Version with CrewAI Data)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Download HTML report from Dropbox and use it with real CrewAI result data

import requests

print("🔄 Loading HTML investment report from Dropbox...")

# Download the HTML report script
dropbox_url = "https://www.dropbox.com/scl/fi/kwuifm4ullucyvyxnnax0/HTML-Investment-Report.html?rlkey=jkfjmxnoks7rxwuxfvh46slap&dl=1"

response = requests.get(dropbox_url)
code_content = response.text

# Execute the code to load the function
exec(code_content)

print(f"🎨 Generating HTML report with CrewAI result data...")

# Pass the result from your earlier CrewAI analysis
html_report = create_html_investment_report(symbol, result)

display(HTML(html_report))

print("✅ HTML Investment Report with real AI recommendations displayed!")

## 🧪 Hands-On Lab: Customize and Explore Your Investment Crew

In this lab, you'll modify your CrewAI investment model by experimenting with agent roles, adding financial tools, and testing different stock symbols. Follow the steps below and submit your observations.

---

### <span style="color:#3b82f6; font-weight:bold;">1. Modify the Strategic Agent</span>
Update the Strategic Agent’s role, goal, or tools to see how it affects the model.
- You might make it more risk-focused or give it access to tools like news search or valuation metrics.

---

### <span style="color:#3b82f6; font-weight:bold;">2. Add a Financial Tool or Calculator</span>
Create a simple helper function or add a tool to compute key financial metrics.
- For example: risk/reward ratio, moving averages, or P/E ratio.

---

### <span style="color:#3b82f6; font-weight:bold;">3. Test New Stock Symbols</span>
Run the model using at least three other stock symbols:
- Suggestions: `TSLA`, `GOOGL`, and `NVDA`. Compare how the recommendations change across different companies.

---

### <span style="color:#3b82f6; font-weight:bold;">4. Submit a 1-Page PDF Report</span>
Write a brief summary covering:What you changed, What symbols you tested, What you observed

Export the report as a **1-2 page PDF**.

---

### <span style="color:#3b82f6; font-weight:bold;">5. Confirm Your Submission</span>
Complete the next cell to finalize your lab submission.
- Make sure your code is saved and your PDF is ready.

---


In [ ]:
# +++++ 🎓 Lab Completion Certificate (Dropbox Version)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Download and run completion certificate from Dropbox

import requests

print("🔄 Loading completion certificate from Dropbox...")

# Download and execute the completion script
dropbox_url = "https://www.dropbox.com/scl/fi/5molmat6myeqaf96kp50v/CrewAI_Completiton.py?rlkey=7v7yaf9gi5hupkxiqaits50rd&dl=1"

response = requests.get(dropbox_url)
exec(response.text)